In [1]:
import polars as pl

In [2]:
train = pl.read_csv('train.csv')
train = train.with_columns(pl.col("first_day_of_month").str.strptime(pl.Date, fmt="%Y-%m-%d"))
train = train.sort(['first_day_of_month','cfips'])

In [3]:
def add_nxt_month_col(df,period = 1,col='microbusiness_density',col2='microbusiness_density'):
    shift_down_value = -df.select('cfips').unique().shape[0] * period

    df = df.with_columns(
        pl.col(col).shift(shift_down_value).alias(f'next_{period}month {col2}')
        )

    return df

def add_lag_month_col(df,period = 1,col='microbusiness_density',col2='microbusiness_density'):
    shift_down_value = df.select('cfips').unique().shape[0] * period

    df = df.with_columns(
        pl.col(col).shift(shift_down_value).alias(f'last_{period}month {col2}')
        )

    return df

In [4]:
# ##microbuisness_density
# #lag
# train = add_nxt_month_col(train,1)
# train = add_nxt_month_col(train,2)
# #lag
# train = add_lag_month_col(train,1)
# train = add_lag_month_col(train,2)

In [5]:
train.filter(pl.col("cfips") == 1001)

row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
str,i64,str,str,date,f64,i64
"""1001_2019-08-0...",1001,"""Autauga County...","""Alabama""",2019-08-01,3.0076818,1249
"""1001_2019-09-0...",1001,"""Autauga County...","""Alabama""",2019-09-01,2.8848701,1198
"""1001_2019-10-0...",1001,"""Autauga County...","""Alabama""",2019-10-01,3.0558431,1269
"""1001_2019-11-0...",1001,"""Autauga County...","""Alabama""",2019-11-01,2.9932332,1243
"""1001_2019-12-0...",1001,"""Autauga County...","""Alabama""",2019-12-01,2.9932332,1243
"""1001_2020-01-0...",1001,"""Autauga County...","""Alabama""",2020-01-01,2.96909,1242
"""1001_2020-02-0...",1001,"""Autauga County...","""Alabama""",2020-02-01,2.9093256,1217
"""1001_2020-03-0...",1001,"""Autauga County...","""Alabama""",2020-03-01,2.9332314,1227
"""1001_2020-04-0...",1001,"""Autauga County...","""Alabama""",2020-04-01,3.0001674,1255


# Census -> Train Format

In [6]:
census = pl.read_csv('census_starter.csv')
census = census.with_columns(pl.col("cfips").cast(pl.Utf8, strict=False))
census.head()

pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64
76.6,78.9,80.6,82.7,85.5,"""1001""",14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
74.5,78.1,81.8,85.1,87.9,"""1003""",20.4,20.7,21.0,20.2,20.6,3.2,3.4,3.7,3.4,3.5,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
57.2,60.4,60.5,64.6,64.6,"""1005""",7.6,7.8,7.6,7.3,6.7,2.7,2.5,2.7,2.6,2.6,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
62.0,66.1,69.2,76.1,74.6,"""1007""",8.1,7.6,6.5,7.4,7.9,1.0,1.4,1.5,1.6,1.1,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
65.8,68.5,73.0,79.6,81.0,"""1009""",8.7,8.1,8.6,8.9,9.3,4.5,4.4,4.5,4.4,4.5,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


In [7]:
census = census.drop('cfips').transpose(include_header=True, column_names=census.get_column("cfips"))

In [8]:
pct_bb = census[0:5, :]
pct_college = census[5:10, :]
pct_foreign = census[10:15, :]
pct_it = census[15:20, :]
median_hh = census[20:25, :]

In [9]:
pct_bb = pct_bb.with_columns(
    pl.col('column').str.replace('pct_bb_','')
)

pct_college = pct_college .with_columns(
    pl.col('column').str.replace('pct_college_','')
)

pct_foreign = pct_foreign.with_columns(
    pl.col('column').str.replace('pct_foreign_born_','')
)

pct_it = pct_it.with_columns(
    pl.col('column').str.replace('pct_it_workers_','')
)

median_hh = median_hh.with_columns(
    pl.col('column').str.replace('median_hh_inc_','')
)

In [10]:
pct_bb.head()

column,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,1021,1023,1025,1027,1029,1031,1033,1035,1037,1039,1041,1043,1045,1047,1049,1051,1053,1055,1057,1059,1061,1063,1065,1067,1069,1071,...,55115,55117,55119,55121,55123,55125,55127,55129,55131,55133,55135,55137,55139,55141,56001,56003,56005,56007,56009,56011,56013,56015,56017,56019,56021,56023,56025,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2017""",76.6,74.5,57.2,62.0,65.8,49.4,58.2,71.0,62.8,67.5,56.6,52.3,47.3,66.0,57.8,75.3,65.9,40.7,58.2,64.7,57.1,65.2,69.8,57.7,61.0,78.2,54.9,68.4,64.1,60.3,64.3,42.0,50.7,68.3,67.6,62.7,...,72.8,79.7,67.5,71.8,66.8,68.8,79.5,71.9,83.7,87.5,73.5,68.9,78.8,75.8,81.0,72.0,86.4,72.5,76.2,70.0,71.9,71.3,74.4,69.2,82.4,76.2,78.5,67.8,80.4,68.4,80.9,82.9,82.2,83.5,83.8,76.4,71.1
"""2018""",78.9,78.1,60.4,66.1,68.5,58.9,62.1,73.0,66.5,68.6,61.8,58.3,53.2,68.2,60.4,77.6,68.2,47.2,64.3,68.6,59.6,69.9,72.9,63.4,67.0,79.6,57.3,73.4,68.9,59.7,66.9,46.2,54.9,72.2,70.4,67.1,...,74.8,81.7,72.6,75.8,70.8,72.6,81.4,74.8,85.0,88.8,76.6,72.4,81.6,78.4,83.0,74.1,89.2,74.8,77.7,72.0,74.6,73.5,75.7,72.0,83.9,79.6,80.5,73.3,82.2,69.2,81.1,81.7,82.4,85.9,88.2,78.3,73.3
"""2019""",80.6,81.8,60.5,69.2,73.0,60.1,64.6,75.1,69.4,70.7,68.7,61.3,57.5,70.1,64.3,80.0,70.1,52.7,69.5,70.7,65.1,74.1,77.0,69.7,72.0,81.5,61.0,76.5,70.3,61.9,67.1,51.3,61.5,75.1,74.9,70.9,...,77.9,83.8,74.4,78.2,72.9,77.1,83.1,76.7,86.9,89.9,79.0,75.1,83.8,81.2,86.4,78.7,91.0,79.4,79.8,73.5,78.0,75.6,77.5,79.3,85.5,83.1,82.5,72.2,85.7,70.9,82.9,85.6,84.0,87.1,89.5,78.2,76.8
"""2020""",82.7,85.1,64.6,76.1,79.6,60.6,73.6,79.8,74.5,75.0,74.6,62.8,62.8,72.0,70.7,82.7,72.6,62.0,75.7,75.2,66.6,78.6,80.9,71.9,78.4,83.7,65.5,80.2,70.3,63.1,70.6,55.6,62.7,78.2,78.0,75.3,...,79.8,86.8,77.9,81.2,74.9,80.3,84.2,80.2,89.3,91.0,82.4,78.6,86.3,83.1,89.0,79.4,91.2,82.0,80.3,79.5,80.1,78.2,81.2,83.7,88.6,88.1,85.8,77.6,86.7,74.5,82.8,88.1,86.7,89.1,91.4,82.8,79.7
"""2021""",85.5,87.9,64.6,74.6,81.0,59.4,76.3,81.6,77.1,76.7,78.3,63.7,67.8,73.4,72.8,83.8,74.5,61.6,75.0,75.1,64.4,81.9,82.3,72.8,79.9,85.4,68.1,82.3,71.9,60.7,72.1,55.8,67.5,80.4,80.6,78.4,...,80.9,88.4,78.5,82.9,77.1,82.3,85.9,80.6,90.5,92.1,83.9,81.3,88.2,85.5,89.7,83.2,91.5,85.4,83.8,82.7,82.5,78.4,83.8,85.4,89.2,89.5,87.6,74.3,87.9,75.1,84.7,89.8,88.4,90.5,90.6,85.4,81.3


In [11]:
def apply_transforms(df,name):
    df = df.with_columns(pl.col('column').str.strptime(pl.Date, fmt='%Y'))
    df = df.upsample(time_column="column", every="1mo").fill_null(strategy="forward")
    df = df.melt(id_vars='column', 
            value_vars=list(df.columns[1:]),
            variable_name='cfips', 
            value_name=name)
    df = df.with_columns([
        pl.col(['column','cfips']).cast(str, strict=False),
        pl.map(["column","cfips"], lambda s: s[1] + "_" + s[0]).alias("row_id"),
        ])
    return df.drop(['column','cfips'])

In [12]:
pct_bb = apply_transforms(pct_bb,'pct_bb')
pct_college = apply_transforms(pct_college,'pct_college')
pct_foreign = apply_transforms(pct_foreign,'pct_foreign')
pct_it = apply_transforms(pct_it,'pct_it')
median_hh =apply_transforms(median_hh,'median_hh')

In [13]:
pct_bb.head()

pct_bb,row_id
f64,str
76.6,"""1001_2017-01-0..."
76.6,"""1001_2017-02-0..."
76.6,"""1001_2017-03-0..."
76.6,"""1001_2017-04-0..."
76.6,"""1001_2017-05-0..."


# JOIN Census with Train

In [14]:
train = train.join(pct_bb, on="row_id", how="outer")
train = train.join(pct_college, on="row_id", how="outer")
train = train.join(pct_foreign, on="row_id", how="outer")
train = train.join(pct_it, on="row_id", how="outer")
train = train.join(median_hh, on="row_id", how="outer")

# GDP Clean

In [15]:
gdp = pl.read_csv("GDP_By_County_2001_2021.csv", infer_schema_length=0)
gdp=gdp.drop(['TableName','LineCode','IndustryClassification','Unit'])
gdp = gdp[:-4,[0,1,2,3,20,21,22,23,24]] #Neg are year cols #Pos are other vars
gdp = gdp.with_columns([
        pl.col('Description').str.strip(),
        pl.col(['Region','2017','2018','2019','2020','2021',]).cast(pl.Int64, strict=False)
        ])

# GDP EDA

# GDP -> Train Format

In [16]:
for i in range(34):#<-this could be paralleled but some errors
    value_name = gdp.select('Description').unique()[i,0]
    df = gdp.filter(pl.col("Description") == value_name)
    df = df.drop(['GeoFIPS','GeoName','Region','Description']).transpose(include_header=True, column_names=df.get_column("GeoFIPS"))
    df = apply_transforms(df, value_name)
    train = train.join(df, on="row_id", how="outer")

# Profile Clean

In [17]:
profile = pl.read_csv("Economic_Profile_By_County_2001_2021.csv", infer_schema_length=0)
profile=profile.drop(['TableName','LineCode','IndustryClassification','Unit'])
profile = profile[:-4,[0,1,2,3,20,21,22,23,24]] #Neg are year cols #Pos are other vars
profile = profile.with_columns([
        pl.col('Description').str.strip(),
        pl.col(['Region','2017','2018','2019','2020','2021',]).cast(pl.Int64, strict=False)
        ])

# Profile EDA

In [18]:
profile

GeoFIPS,GeoName,Region,Description,2017,2018,2019,2020,2021
str,str,i64,str,i64,i64,i64,i64,i64
"""0""","""United States""",null,"""Personal incom...",16837337000,17671054000,18575467000,19812171000,21288709000
"""0""","""United States""",null,"""Net earnings b...",10631149000,11157678000,11619305000,11771316000,12750109000
"""0""","""United States""",null,"""Personal curre...",2855645000,2976552000,3144768000,4231121000,4617314000
"""0""","""United States""",null,"""Income mainten...",269785000,263267000,273295000,315649000,473586000
"""0""","""United States""",null,"""Unemployment i...",30634000,28155000,27948000,537713000,321127000
"""0""","""United States""",null,"""Retirement and...",2555226000,2685130000,2843525000,3377759000,3822601000
"""0""","""United States""",null,"""Dividends, int...",3350543000,3536824000,3811394000,3809734000,3921286000
"""0""","""United States""",null,"""Population (pe...",326623063,328542157,330233102,331501080,331893745
"""0""","""United States""",null,"""Per capita per...",51550,53786,56250,59765,64143


# Profile -> Train Format

In [19]:
for i in range(31):#<-this could be paralleled but some errors
    value_name = profile.select('Description').unique()[i,0]
    df = profile.filter(pl.col("Description") == value_name)
    df = df.drop(['GeoFIPS','GeoName','Region','Description']).transpose(include_header=True, column_names=df.get_column("GeoFIPS"))
    df = apply_transforms(df, value_name)
    train = train.join(df, on="row_id", how="outer")

In [26]:
train.sort(['row_id'])

row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb,pct_college,pct_foreign,pct_it,median_hh,All industry total,Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Finance and insurance,Real estate and rental and leasing,Professional and business services,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,"Educational services, health care, and social assistance",Educational services,Health care and social assistance,"Arts, entertainment, recreation, accommodation, and food services","Arts, entertainment, and recreation",...,Natural resources and mining,Trade,Transportation and utilities,Manufacturing and information,Private goods-producing industries 2/,Private services-providing industries 3/,Personal income (thousands of dollars),Net earnings by place of residence,Personal current transfer receipts,Income maintenance benefits 1/,Unemployment insurance compensation,Retirement and other,"Dividends, interest, and rent 2/",Population (persons) 3/,Per capita personal income 4/,Per capita net earnings 4/,Per capita personal current transfer receipts 4/,Per capita income maintenance benefits 4/,Per capita unemployment insurance compensation 4/,Per capita retirement and other 4/,"Per capita dividends, interest, and rent 4/",Earnings by place of work,Wages and salaries,Supplements to wages and salaries,Employer contributions for employee pension and insurance funds 5/,Employer contributions for government social insurance,Proprietors' income,Farm proprietors' income,Nonfarm proprietors' income,Total employment (number of jobs),Wage and salary employment,Proprietors employment,Farm proprietors employment 6/,Nonfarm proprietors employment,Average earnings per job (dollars),Average wages and salaries,Average nonfarm proprietors' income
str,i64,str,str,date,f64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,...,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""0_2017-01-01""",null,null,null,null,null,null,null,null,null,null,null,19477337000,17031697000,175433000,275397000,311556000,800633000,2199664000,1235637000,964027000,1165730000,1081874000,609083000,1004215000,4019950000,1512330000,2507621000,2434937000,1458772000,372830000,603334000,1711179000,249292000,1461887000,828668000,217808000,...,450830000,2247603000,920638000,3203880000,3451127000,13580569000,16837337000,10631149000,2855645000,269785000,30634000,2555226000,3350543000,326623063,51550,32549,8743,826,94,7823,10258,11925461000,8469518000,1944272000,1340434000,603838000,1511671000,46150000,1465521000,196394100,151949000,44445100,1803000,42642100,60722,55739,34368
"""0_2017-02-01""",null,null,null,null,null,null,null,null,null,null,null,19477337000,17031697000,175433000,275397000,311556000,800633000,2199664000,1235637000,964027000,1165730000,1081874000,609083000,1004215000,4019950000,1512330000,2507621000,2434937000,1458772000,372830000,603334000,1711179000,249292000,1461887000,828668000,217808000,...,450830000,2247603000,920638000,3203880000,3451127000,13580569000,16837337000,10631149000,2855645000,269785000,30634000,2555226000,3350543000,326623063,51550,32549,8743,826,94,7823,10258,11925461000,8469518000,1944272000,1340434000,603838000,1511671000,46150000,1465521000,196394100,151949000,44445100,1803000,42642100,60722,55739,34368
"""0_2017-03-01""",null,null,null,null,null,null,null,null,null,null,null,19477337000,17031697000,175433000,275397000,311556000,800633000,2199664000,12356370

# 
Export Train file

In [28]:
train.write_csv('train+.csv')